## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-02-10-46-06 +0000,nyt,Snow Falls in New York State as Major Storm Ap...,https://www.nytimes.com/2025/12/02/us/snow-win...
1,2025-12-02-10-42-46 +0000,bbc,Ukraine in maps: Tracking the war with Russia,https://www.bbc.com/news/articles/c0l0k4389g2o...
2,2025-12-02-10-42-21 +0000,bbc,UK faces 'headwind' from tax and spending move...,https://www.bbc.com/news/articles/cr5zdeyd1eeo...
3,2025-12-02-10-41-00 +0000,wsj,Euro Remains Steady After Eurozone Inflation Data,https://www.wsj.com/finance/currencies/bitcoin...
4,2025-12-02-10-37-01 +0000,bbc,Nigeria grants asylum to Guinea-Bissau preside...,https://www.bbc.com/news/articles/ce3wq2zznz1o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2351/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
299,trump,52
125,house,23
2,new,22
307,white,15
68,up,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
130,2025-12-02-00-36-00 +0000,wsj,The White House published a website detailing ...,https://www.wsj.com/politics/policy/new-white-...,139
193,2025-12-01-21-10-44 +0000,nypost,White House doesn’t rule out boots on the grou...,https://nypost.com/2025/12/01/us-news/trump-do...,121
209,2025-12-01-20-17-00 +0000,wsj,The White House said Trump underwent advanced ...,https://www.wsj.com/politics/white-house-says-...,117
89,2025-12-02-03-20-00 +0000,wsj,Costco one of the biggest companies to sue the...,https://www.wsj.com/politics/policy/costco-joi...,114
243,2025-12-01-17-58-00 +0000,wsj,Trump pushed voters to cast a ballot for the R...,https://www.wsj.com/politics/elections/tenness...,111


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
130,139,2025-12-02-00-36-00 +0000,wsj,The White House published a website detailing ...,https://www.wsj.com/politics/policy/new-white-...
80,57,2025-12-02-04-46-00 +0000,wsj,The White House said the U.S. conducted two st...,https://www.wsj.com/politics/white-house-says-...
89,53,2025-12-02-03-20-00 +0000,wsj,Costco one of the biggest companies to sue the...,https://www.wsj.com/politics/policy/costco-joi...
93,50,2025-12-02-03-11-29 +0000,nyt,Kristi Noem Recommends New Travel Ban After Na...,https://www.nytimes.com/2025/12/01/us/politics...
81,43,2025-12-02-04-45-30 +0000,nypost,Ryan Lizza claims ex Olivia Nuzzi and RFK Jr. ...,https://nypost.com/2025/12/01/us-news/ryan-liz...
299,39,2025-12-01-12-20-44 +0000,nypost,Hong Kong arrests more suspects in fire probe ...,https://nypost.com/2025/12/01/world-news/hong-...
127,38,2025-12-02-00-45-32 +0000,nypost,Little-known ‘Christmas Tree Law’ bans sidewal...,https://nypost.com/2025/12/01/us-news/little-k...
314,37,2025-12-01-11-00-00 +0000,latimes,"Attacks on ICE up 1,000%? Trump administration...",https://www.latimes.com/california/story/2025-...
91,37,2025-12-02-03-13-14 +0000,nypost,Trump calls Tennessee special election a ‘big ...,https://nypost.com/2025/12/01/us-news/trump-ca...
13,33,2025-12-02-10-12-41 +0000,wapo,U.S. delegation to meet Putin in latest bid to...,https://www.washingtonpost.com/world/2025/12/0...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
